# LSTM Model - Emotion and Empathy Prediction

## Setup and Imports

In [5]:
!pip install nltk
!pip install pandas
!pip install gensim
!pip install re
!pip install numpy
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [6]:
import gensim
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime

In [7]:
##################################
# This cell works only on colab  #
##################################
import sys
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
DRIVE_NAME = "NLPproject1"
ROOT_PATH = os.path.join('/content/drive/MyDrive/', DRIVE_NAME)
SCRIPTS_PATH = f'{ROOT_PATH}/Scripts'
DATA_PATH = f'{ROOT_PATH}/Dataset'

if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)

from preprocessing import GloveSequenceEmbedder
from dataset import RNNEmpathyDataset, RNNInferenceDataset
from lstm_model import LSTMEmpathyNet

print("Setup completed")
print(f"Root Path: {ROOT_PATH}")
print(f"Scripts Path: {SCRIPTS_PATH}")
print(f"Data Path: {DATA_PATH}")

Mounted at /content/drive
Setup completed
Root Path: /content/drive/MyDrive/NLPproject1
Scripts Path: /content/drive/MyDrive/NLPproject1/Scripts
Data Path: /content/drive/MyDrive/NLPproject1/Dataset


## Initialize GloVe Sequence Embedder

In [8]:
print("Initializing GloVe Sequence Embedder...")
sequence_embedder = GloveSequenceEmbedder(
    model_name="glove-wiki-gigaword-100",
    max_length=50
)

print(f"Sequence Embedder initialized")
print(f"Vector size: {sequence_embedder.vector_size}")
print(f"Max length: {sequence_embedder.max_length}")

Initializing GloVe Sequence Embedder...
Loading GloVe model 'glove-wiki-gigaword-100' for sequence embedding...
[==================================================] 100.0% 128.1/128.1MB downloaded
GloVe sequence model loaded successfully. Max length: 50
Sequence Embedder initialized
Vector size: 100
Max length: 50


## Loss Functions

In [9]:
regression_loss = nn.MSELoss()
classification_loss = nn.CrossEntropyLoss()

loss_weights = {
    'intensity': 0.2,
    'empathy': 0.2,
    'polarity': 0.6
}

loss_functions = {
    'regression': regression_loss,
    'classification': classification_loss
}

print("Loss functions configured")

Loss functions configured


## Data Preparation

In [13]:
train_csv_path = f"{DATA_PATH}/trac2_CONVT_train.csv"
eval_csv_path = f"{DATA_PATH}/trac2_CONVT_dev.csv"

print("Creating LSTM datasets...")
lstm_train_dataset = RNNEmpathyDataset(train_csv_path, sequence_embedder)
lstm_eval_dataset = RNNEmpathyDataset(eval_csv_path, sequence_embedder)

LSTM_BATCH_SIZE = 32
lstm_train_loader = DataLoader(lstm_train_dataset, batch_size=LSTM_BATCH_SIZE, shuffle=True)
lstm_eval_loader = DataLoader(lstm_eval_dataset, batch_size=LSTM_BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(lstm_train_dataset)}")
print(f"Evaluation samples: {len(lstm_eval_dataset)}")

Creating LSTM datasets...
Loaded and cleaned data from /content/drive/MyDrive/NLPproject1/Dataset/trac2_CONVT_train.csv. Number of samples: 11090
Generating sequence embeddings for 11090 samples...
Loaded and cleaned data from /content/drive/MyDrive/NLPproject1/Dataset/trac2_CONVT_dev.csv. Number of samples: 987
Generating sequence embeddings for 987 samples...
Training samples: 11090
Evaluation samples: 987


## Model Configuration

In [12]:
lstm_experiment_configs = [
    {
        "experiment_name": "lstm_baseline_h128_d0.3",
        "input_dim": sequence_embedder.vector_size,
        "hidden_dim": 128,
        "num_layers": 2,
        "bidirectional": True,
        "dropout": 0.3,
        "num_classes": 4,
        "learning_rate": 1e-3,
    },
    {
        "experiment_name": "lstm_deep_h256_d0.5",
        "input_dim": sequence_embedder.vector_size,
        "hidden_dim": 256,
        "num_layers": 3,
        "bidirectional": True,
        "dropout": 0.5,
        "num_classes": 4,
        "learning_rate": 5e-4,
    },
    {
        "experiment_name": "lstm_unidirectional_h128_d0.2",
        "input_dim": sequence_embedder.vector_size,
        "hidden_dim": 128,
        "num_layers": 2,
        "bidirectional": False,
        "dropout": 0.2,
        "num_classes": 4,
        "learning_rate": 1e-3,
    },
    {
        "experiment_name": "lstm_wide_h512_d0.4",
        "input_dim": sequence_embedder.vector_size,
        "hidden_dim": 512,
        "num_layers": 2,
        "bidirectional": True,
        "dropout": 0.4,
        "num_classes": 4,
        "learning_rate": 1e-4,
    }
]

NUM_LSTM_EPOCHS = 15
LSTM_MODELS_SAVE_PATH = f"{ROOT_PATH}/Saved Models/LSTM"
os.makedirs(LSTM_MODELS_SAVE_PATH, exist_ok=True)

print(f"Configured {len(lstm_experiment_configs)} LSTM experiments")

Configured 4 LSTM experiments


## Training Functions

In [21]:
def train_lstm_one_epoch(model, dataloader, optimizer, loss_fns, device):
    model.train()
    total_loss = 0.0
    loss_weights = {'intensity': 0.2, 'empathy': 0.2, 'polarity': 0.6}

    for batch in dataloader:
        features = batch['features'].to(device)
        # Calculate lengths based on features (assuming 0 is padding)
        lengths = (features.sum(dim=2) != 0).sum(dim=1).to(device)
        # Ensure lengths are at least 1
        lengths = torch.clamp(lengths, min=1)
        labels = {k: v.to(device) for k, v in batch['labels'].items()}

        outputs = model(features, lengths)

        loss_intensity = loss_fns['regression'](outputs['intensity'], labels['intensity'])
        loss_empathy = loss_fns['regression'](outputs['empathy'], labels['empathy'])
        loss_polarity = loss_fns['classification'](outputs['polarity'], labels['polarity'])

        combined_loss = (loss_weights['intensity'] * loss_intensity +
                        loss_weights['empathy'] * loss_empathy +
                        loss_weights['polarity'] * loss_polarity)

        optimizer.zero_grad()
        combined_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        total_loss += combined_loss.item()

    return total_loss / len(dataloader)


def evaluate_lstm_performance(model, dataloader, loss_fns, device):
    model.eval()
    total_loss = 0.0
    loss_weights = {'intensity': 0.2, 'empathy': 0.2, 'polarity': 0.6}

    all_intensity_preds, all_intensity_labels = [], []
    all_empathy_preds, all_empathy_labels = [], []
    all_polarity_preds, all_polarity_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            features = batch['features'].to(device)
            # Calculate lengths based on features (assuming 0 is padding)
            lengths = (features.sum(dim=2) != 0).sum(dim=1).to(device)
            # Ensure lengths are at least 1
            lengths = torch.clamp(lengths, min=1)
            labels = {k: v.to(device) for k, v in batch['labels'].items()}

            outputs = model(features, lengths)

            loss_intensity = loss_fns['regression'](outputs['intensity'], labels['intensity'])
            loss_empathy = loss_fns['regression'](outputs['empathy'], labels['empathy'])
            loss_polarity = loss_fns['classification'](outputs['polarity'], labels['polarity'])
            combined_loss = (loss_weights['intensity'] * loss_intensity +
                            loss_weights['empathy'] * loss_empathy +
                            loss_weights['polarity'] * loss_polarity)
            total_loss += combined_loss.item()

            all_intensity_preds.append(outputs['intensity'].cpu())
            all_intensity_labels.append(labels['intensity'].cpu())
            all_empathy_preds.append(outputs['empathy'].cpu())
            all_empathy_labels.append(labels['empathy'].cpu())

            polarity_preds = torch.argmax(outputs['polarity'], dim=1)
            all_polarity_preds.append(polarity_preds.cpu())
            all_polarity_labels.append(labels['polarity'].cpu())

    all_intensity_preds = torch.cat(all_intensity_preds)
    all_intensity_labels = torch.cat(all_intensity_labels)
    all_empathy_preds = torch.cat(all_empathy_preds)
    all_empathy_labels = torch.cat(all_empathy_labels)
    all_polarity_preds = torch.cat(all_polarity_preds)
    all_polarity_labels = torch.cat(all_polarity_labels)

    mae_intensity = nn.functional.l1_loss(all_intensity_preds, all_intensity_labels).item()
    mae_empathy = nn.functional.l1_loss(all_empathy_preds, all_empathy_labels).item()

    accuracy_polarity = accuracy_score(all_polarity_labels, all_polarity_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_polarity_labels, all_polarity_preds, average='weighted', zero_division=0
    )

    metrics = {
        "val_loss": total_loss / len(dataloader),
        "intensity_mae": mae_intensity,
        "empathy_mae": mae_empathy,
        "polarity_accuracy": accuracy_polarity,
        "polarity_precision": precision,
        "polarity_recall": recall,
        "polarity_f1": f1
    }

    return metrics

## Training Loop

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

lstm_results = []
best_overall_val_loss = float('inf')

for config in lstm_experiment_configs:
    print(f"\n{'='*50}")
    print(f"Experiment: {config['experiment_name']}")

    model = LSTMEmpathyNet(config).to(device)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Parameters: {total_params:,}")

    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    best_epoch_metrics = {"val_loss": float('inf')}
    patience_counter = 0
    early_stopping_patience = 5

    for epoch in range(NUM_LSTM_EPOCHS):
        train_loss = train_lstm_one_epoch(model, lstm_train_loader, optimizer, loss_functions, device)
        val_metrics = evaluate_lstm_performance(model, lstm_eval_loader, loss_functions, device)

        scheduler.step(val_metrics['val_loss'])

        print(f"Epoch {epoch+1}/{NUM_LSTM_EPOCHS} -> Train: {train_loss:.4f} | Val: {val_metrics['val_loss']:.4f} | F1: {val_metrics['polarity_f1']:.4f}")

        if val_metrics['val_loss'] < best_epoch_metrics['val_loss']:
            best_epoch_metrics = val_metrics
            model_save_path = os.path.join(LSTM_MODELS_SAVE_PATH, f"{config['experiment_name']}_best.pth")
            torch.save(model.state_dict(), model_save_path)
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    final_result = {
        "experiment_name": config['experiment_name'],
        "model_path": model_save_path,
        "hidden_dim": config['hidden_dim'],
        "num_layers": config['num_layers'],
        "bidirectional": config['bidirectional'],
        "dropout": config['dropout'],
        "learning_rate": config['learning_rate'],
        "total_params": total_params
    }
    final_result.update(best_epoch_metrics)
    lstm_results.append(final_result)

    if best_epoch_metrics['val_loss'] < best_overall_val_loss:
        best_overall_val_loss = best_epoch_metrics['val_loss']
        print(f"New best model: {config['experiment_name']}")

print(f"\n{'='*50}")
print(f"Best validation loss: {best_overall_val_loss:.4f}")

Using device: cpu

Experiment: lstm_baseline_h128_d0.3
Parameters: 632,326
Epoch 1/15 -> Train: 0.8398 | Val: 0.7671 | F1: 0.6032
Epoch 2/15 -> Train: 0.6924 | Val: 0.7335 | F1: 0.6498
Epoch 3/15 -> Train: 0.6567 | Val: 0.7100 | F1: 0.6712
Epoch 4/15 -> Train: 0.6379 | Val: 0.7158 | F1: 0.6757
Epoch 5/15 -> Train: 0.6196 | Val: 0.7000 | F1: 0.6815
Epoch 6/15 -> Train: 0.6030 | Val: 0.7112 | F1: 0.6588
Epoch 7/15 -> Train: 0.5877 | Val: 0.7160 | F1: 0.6536
Epoch 8/15 -> Train: 0.5684 | Val: 0.7099 | F1: 0.6754
Epoch 9/15 -> Train: 0.5311 | Val: 0.7313 | F1: 0.6652
Epoch 10/15 -> Train: 0.5134 | Val: 0.7527 | F1: 0.6542
Early stopping at epoch 10
New best model: lstm_baseline_h128_d0.3

Experiment: lstm_deep_h256_d0.5
Parameters: 3,890,182
Epoch 1/15 -> Train: 0.8621 | Val: 0.7742 | F1: 0.6019
Epoch 2/15 -> Train: 0.7046 | Val: 0.7595 | F1: 0.6317
Epoch 3/15 -> Train: 0.6765 | Val: 0.7279 | F1: 0.6457
Epoch 4/15 -> Train: 0.6569 | Val: 0.7155 | F1: 0.6472
Epoch 5/15 -> Train: 0.6437 | Va

## Results Analysis

In [23]:
lstm_results_df = pd.DataFrame(lstm_results)

lstm_display_columns = {
    'experiment_name': 'Experiment',
    'hidden_dim': 'Hidden',
    'num_layers': 'Layers',
    'bidirectional': 'Bidir',
    'dropout': 'Dropout',
    'val_loss': 'Val Loss',
    'intensity_mae': 'Int MAE',
    'empathy_mae': 'Emp MAE',
    'polarity_f1': 'Pol F1'
}

lstm_results_display = lstm_results_df[list(lstm_display_columns.keys())].rename(columns=lstm_display_columns)
lstm_results_sorted = lstm_results_display.sort_values(by="Val Loss", ascending=True).reset_index(drop=True)

REPORT_PATH = os.path.join(ROOT_PATH, 'Report')
os.makedirs(REPORT_PATH, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d")
lstm_report_filepath = os.path.join(REPORT_PATH, f"lstm_results_{timestamp}.csv")
lstm_results_sorted.to_csv(lstm_report_filepath, index=False)

print("LSTM Experiment Results:")
print(lstm_results_sorted.to_string(index=False))

best_experiment_name = lstm_results_sorted.iloc[0]['Experiment']
best_experiment_full_info = next(item for item in lstm_results if item["experiment_name"] == best_experiment_name)

print(f"\nBest model: {best_experiment_full_info['experiment_name']}")
print(f"Val Loss: {best_experiment_full_info['val_loss']:.4f}")
print(f"Intensity MAE: {best_experiment_full_info['intensity_mae']:.4f}")
print(f"Empathy MAE: {best_experiment_full_info['empathy_mae']:.4f}")
print(f"Polarity F1: {best_experiment_full_info['polarity_f1']:.4f}")

LSTM Experiment Results:
                   Experiment  Hidden  Layers  Bidir  Dropout  Val Loss  Int MAE  Emp MAE   Pol F1
lstm_unidirectional_h128_d0.2     128       2  False      0.2  0.681776 0.500202 0.738660 0.675564
          lstm_wide_h512_d0.4     512       2   True      0.4  0.699658 0.493168 0.743910 0.678621
      lstm_baseline_h128_d0.3     128       2   True      0.3  0.700002 0.490881 0.755700 0.681496
          lstm_deep_h256_d0.5     256       3   True      0.5  0.701584 0.500063 0.742865 0.665044

Best model: lstm_unidirectional_h128_d0.2
Val Loss: 0.6818
Intensity MAE: 0.5002
Empathy MAE: 0.7387
Polarity F1: 0.6756


## Evaluation on Dev Set

In [27]:
best_model_config = next(config for config in lstm_experiment_configs
                         if config["experiment_name"] == best_experiment_name)
best_lstm_model = LSTMEmpathyNet(best_model_config).to(device)
best_lstm_model.load_state_dict(torch.load(best_experiment_full_info['model_path']))
best_lstm_model.eval()

all_intensity_preds, all_intensity_labels = [], []
all_empathy_preds, all_empathy_labels = [], []
all_polarity_preds, all_polarity_labels = [], []

with torch.no_grad():
    for batch in lstm_eval_loader:
        features = batch['features'].to(device)
        # Calculate lengths based on features (assuming 0 is padding)
        lengths = (features.sum(dim=2) != 0).sum(dim=1).to(device)
        # Ensure lengths are at least 1
        lengths = torch.clamp(lengths, min=1)
        labels = {k: v.to(device) for k, v in batch['labels'].items()}

        outputs = best_lstm_model(features, lengths)

        all_intensity_preds.append(outputs['intensity'].cpu())
        all_intensity_labels.append(labels['intensity'].cpu())
        all_empathy_preds.append(outputs['empathy'].cpu())
        all_empathy_labels.append(labels['empathy'].cpu())

        polarity_preds = torch.argmax(outputs['polarity'], dim=1)
        all_polarity_preds.append(polarity_preds.cpu())
        all_polarity_labels.append(labels['polarity'].cpu())

all_intensity_preds = torch.cat(all_intensity_preds).numpy()
all_intensity_labels = torch.cat(all_intensity_labels).numpy()
all_empathy_preds = torch.cat(all_empathy_preds).numpy()
all_empathy_labels = torch.cat(all_empathy_labels).numpy()
all_polarity_preds = torch.cat(all_polarity_preds).numpy()
all_polarity_labels = torch.cat(all_polarity_labels).numpy()

print("REGRESSION METRICS")
print(f"\nEmotion Intensity:")
print(f"  MAE: {mean_absolute_error(all_intensity_labels, all_intensity_preds):.4f}")
print(f"  MSE: {mean_squared_error(all_intensity_labels, all_intensity_preds):.4f}")
print(f"  R²: {r2_score(all_intensity_labels, all_intensity_preds):.4f}")

print(f"\nEmpathy:")
print(f"  MAE: {mean_absolute_error(all_empathy_labels, all_empathy_preds):.4f}")
print(f"  MSE: {mean_squared_error(all_empathy_labels, all_empathy_preds):.4f}")
print(f"  R²: {r2_score(all_empathy_labels, all_empathy_preds):.4f}")

print(f"\nCLASSIFICATION METRICS")
print(f"\nEmotional Polarity:")
polarity_accuracy = accuracy_score(all_polarity_labels, all_polarity_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_polarity_labels, all_polarity_preds, average='weighted', zero_division=0)
print(f"  Accuracy: {polarity_accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")
print(f"  F1-Score: {f1:.4f}")

print(f"\nConfusion Matrix:")
cm = confusion_matrix(all_polarity_labels, all_polarity_preds)
for i in range(cm.shape[0]):
    print(f"  {cm[i]}")

print(f"\nClassification Report:")
# Assuming the classes present are 0, 1, 2 based on the confusion matrix size
present_classes = sorted(list(set(all_polarity_labels).union(set(all_polarity_preds))))
target_names = [f'Class {c}' for c in present_classes]
print(classification_report(all_polarity_labels, all_polarity_preds,
                           target_names=target_names, labels=present_classes, zero_division=0))

REGRESSION METRICS

Emotion Intensity:
  MAE: 0.5002
  MSE: 0.4325
  R²: 0.2262

Empathy:
  MAE: 0.7387
  MSE: 0.8108
  R²: 0.3253

CLASSIFICATION METRICS

Emotional Polarity:
  Accuracy: 0.6859
  Precision: 0.6973
  Recall: 0.6859
  F1-Score: 0.6756

Confusion Matrix:
  [54 87 16]
  [ 14 360  74]
  [  5 114 263]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.74      0.34      0.47       157
     Class 1       0.64      0.80      0.71       448
     Class 2       0.75      0.69      0.72       382

    accuracy                           0.69       987
   macro avg       0.71      0.61      0.63       987
weighted avg       0.70      0.69      0.68       987



## Test Predictions

In [31]:
TEST_CSV_PATH = f"{DATA_PATH}/trac2_CONVT_test.csv"
LSTM_SUBMISSION_PATH = os.path.join(REPORT_PATH, 'lstm_report.csv')

print("Running predictions on test set...")

test_dataset = RNNInferenceDataset(
    TEST_CSV_PATH,
    sequence_embedder,
    id_column='id',
    text_column='text'
)
test_loader = DataLoader(test_dataset, batch_size=LSTM_BATCH_SIZE, shuffle=False)

all_ids = []
all_emotion_preds = []
all_empathy_preds = []
all_polarity_preds = []

with torch.no_grad():
    for batch in test_loader:
        ids = batch['id']
        features = batch['features'].to(device)
        # Calculate lengths based on features (assuming 0 is padding)
        lengths = (features.sum(dim=2) != 0).sum(dim=1).to(device)
        # Ensure lengths are at least 1
        lengths = torch.clamp(lengths, min=1)

        outputs = best_lstm_model(features, lengths)

        emotion_preds = outputs['intensity'].squeeze().cpu().numpy()
        empathy_preds = outputs['empathy'].squeeze().cpu().numpy()
        polarity_preds = torch.argmax(outputs['polarity'], dim=1).cpu().numpy()

        all_ids.extend(ids.numpy())
        all_emotion_preds.extend(emotion_preds)
        all_empathy_preds.extend(empathy_preds)
        all_polarity_preds.extend(polarity_preds)

submission_df = pd.DataFrame({
    'id': all_ids,
    'Emotion': all_emotion_preds,
    'EmotionalPolarity': all_polarity_preds,
    'Empathy': all_empathy_preds
})

submission_df['EmotionalPolarity'] = submission_df['EmotionalPolarity'].astype(int)
submission_df.to_csv(LSTM_SUBMISSION_PATH, index=False)

print(f"Submission file created: {LSTM_SUBMISSION_PATH}")
print(f"\nPreview:")
print(submission_df.head(10))

Running predictions on test set...
Generating sequence embeddings for 2311 test samples...
Submission file created: /content/drive/MyDrive/NLPproject1/Report/lstm_report.csv

Preview:
   id   Emotion  EmotionalPolarity   Empathy
0   1  2.175447                  1  1.911928
1   2  2.437121                  2  2.494642
2   3  2.269280                  2  2.163032
3   4  2.258986                  2  2.234717
4   5  2.344783                  2  2.296820
5   6  2.815558                  2  2.662120
6   7  1.875809                  1  2.007888
7   8  2.217818                  1  2.184005
8   9  1.969614                  1  1.774307
9  10  1.852713                  1  1.812997
